In [1]:
import random
import numpy as np
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


**Load Datasets:**

In [2]:
dataset = load_dataset("imdb")

train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]

test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

**BALANCE THE DATA:**

In [3]:
def balance_dataset(texts, labels, samples_per_class):
    pos = [(t,l) for t,l in zip(texts, labels) if l == 1]
    neg = [(t,l) for t,l in zip(texts, labels) if l == 0]

    random.shuffle(pos)
    random.shuffle(neg)

    pos = pos[:samples_per_class]
    neg = neg[:samples_per_class]

    data = pos + neg
    random.shuffle(data)

    texts, labels = zip(*data)
    return list(texts), list(labels)

# Balanced & CPU-safe
train_texts, train_labels = balance_dataset(train_texts, train_labels, 600)
test_texts, test_labels = balance_dataset(test_texts, test_labels, 300)

print(len(train_texts), sum(train_labels))


1200 600


**Tokenization:**

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    list(test_texts),
    truncation=True,
    padding=True,
    max_length=128
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

**Dataset Class:**

In [5]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)


**Load BERT Model**

In [6]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 **Training Arguments:**

In [7]:
training_args = TrainingArguments(
    output_dir="./bert_cpu",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="no",
    report_to="none"
)


**Metrics Function:**

In [8]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
        "f1": f1_score(labels, preds, zero_division=0)
    }


**Trainer:**

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


**Train Model:**

In [10]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.440427,0.853333,0.841935,0.870000,0.855738
2,0.456800,0.641545,0.850000,0.850000,0.850000,0.850000
3,0.456800,0.733395,0.850000,0.840909,0.863333,0.851974


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=900, training_loss=0.32821585761176214, metrics={'train_runtime': 5972.4649, 'train_samples_per_second': 0.603, 'train_steps_per_second': 0.151, 'total_flos': 236799949824000.0, 'train_loss': 0.32821585761176214, 'epoch': 3.0})

**Final Evaluation:**

In [14]:
results = trainer.evaluate()

results


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.7333953380584717,
 'eval_accuracy': 0.85,
 'eval_precision': 0.8409090909090909,
 'eval_recall': 0.8633333333333333,
 'eval_f1': 0.8519736842105263,
 'eval_runtime': 307.2224,
 'eval_samples_per_second': 1.953,
 'eval_steps_per_second': 0.488,
 'epoch': 3.0}

**BERT :Final Evaluation Metrics.**

In [16]:
import pandas as pd

comparison_df = pd.DataFrame({

    "Accuracy": [results["eval_accuracy"]],
    "Precision": [results["eval_precision"]],
    "Recall": [results["eval_recall"]],
    "F1-Score": [results["eval_f1"]]
})

comparison_df


,Accuracy,Precision,Recall,F1-Score
0,0.85,0.840909,0.863333,0.851974
